<a href="https://colab.research.google.com/github/cipz/AlgoritmiAvanzati/blob/master/Homework2/exactHeldKarp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Held and Karp algorithm

In [247]:
!pip install ipython-autotime

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 3.04 s


In [248]:
import math
import numpy as np
# u, v: vertexes/coordinates
# t: WEIGHT_TYPE
def weight (u, v, t):
  if t == 'EUC_2D':
    return round(math.sqrt(sum([(a - b) ** 2 for a, b in zip(u, v)])))
  else:
    PI = 3.141592
    deg_xu = int(u[0])
    min_xu = u[0] - deg_xu
    rad_xu = PI * (deg_xu + 5.0 * min_xu/ 3.0) / 180.0

    deg_yu = int(u[1])
    min_yu = u[1] - deg_yu
    rad_yu = PI * (deg_yu + 5.0 * min_yu/ 3.0) / 180.0

    deg_xv = int(v[0])
    min_xv = v[0] - deg_xv
    rad_xv= PI * (deg_xv + 5.0 * min_xv/ 3.0) / 180.0

    deg_yv = int(v[1])
    min_yv = v[1]- deg_yv
    rad_yv = PI * (deg_yv + 5.0 * min_yv/ 3.0) / 180.0

    RRR = 6378.388
    q1 = math.cos(rad_yu - rad_yv)
    q2 = math.cos(rad_xu - rad_xv)
    q3 = math.cos(rad_xu + rad_xv)
    return (int) (RRR * math.acos(0.5 * ((1.0 + q1) * q2 - (1.0 - q1) * q3)) + 1.0)

time: 14.9 ms


In [249]:
lines = open("burma14.tsp", "r").readlines()
index_start_coordinates = 0
cont = 0
V = []

for line in lines:
  cont += 1
  if line.startswith("EOF"):
    break
  elif line.startswith("DIMENSION"):
    n = int(line.split(":")[1][1:])
  elif line.startswith("EDGE_WEIGHT_TYPE"):
    t = line.split(":")[1][1:]
  elif line.startswith("NODE_COORD_SECTION"):
    index_start_coordinates = cont
  elif (index_start_coordinates > 0):
    V.append((int(line.split()[0]) - 1, [float(line.split()[1]), float(line.split()[2])])) #(i, [x_value, y_value])
#n = int(lines[3].split()[1]) #.split()[0] # extract number of vertexes
#t = lines[4].split()[1]
print(n,t)

print(t,"EUC_2D")
print(t == "EUC_2D")
print(V)

14 GEO

GEO
 EUC_2D
False
[(0, [16.47, 96.1]), (1, [16.47, 94.44]), (2, [20.09, 92.54]), (3, [22.39, 93.37]), (4, [25.23, 97.24]), (5, [22.0, 96.05]), (6, [20.47, 97.02]), (7, [17.2, 96.29]), (8, [16.3, 97.38]), (9, [14.05, 98.12]), (10, [16.53, 97.38]), (11, [21.52, 95.59]), (12, [19.41, 97.13]), (13, [20.09, 94.55])]
3194.0
3346.0
3703.0
3899.0
4159.0
3774.0
3648.0
3263.0
3353.0
3565.0
3350.0
3760.0
3535.0
3591.0
3346.0
3194.0
3615.0
3857.0
4190.0
3791.0
3700.0
3390.0
3504.0
3672.0
3503.0
3774.0
3610.0
3569.0
3703.0
3615.0
3194.0
3482.0
3937.0
3583.0
3630.0
3684.0
3838.0
4073.0
3811.0
3567.0
3648.0
3404.0
3899.0
3857.0
3482.0
3194.0
3684.0
3458.0
3603.0
3857.0
3997.0
4263.0
3961.0
3452.0
3692.0
3503.0
4159.0
4190.0
3937.0
3684.0
3194.0
3593.0
3707.0
4095.0
4183.0
4454.0
4140.0
3611.0
3828.0
3829.0
3774.0
3791.0
3583.0
3458.0
3593.0
3194.0
3361.0
3715.0
3827.0
4103.0
3786.0
3212.0
3477.0
3432.0
3648.0
3700.0
3630.0
3603.0
3707.0
3361.0
3194.0
3582.0
3675.0
3950.0
3632.0
3356.0
3317.0


In [250]:
# global values

subsets = []
subsets.append(V)
diction = {}
counter = 0

def encode(arr):
  string = ""
  for v in arr:
    string += str(v[0])
  return string

diction[encode(V)] = counter
counter += 1

#phi = [[None] * 10] * 10 #[[]]
#d = [[None] * 10] * 10
d = np.zeros(shape = (n, 2 ** (n - 1))) # 2^(n-1)
phi = np.zeros(shape = (n, 2 ** (n - 1)))

time: 12.7 ms


In [251]:
import time
start = time.time()
def HK_VISIT (v, S): # v: arrival vertex of S, S: subset of vertexes 
  S_index = subsets.index(S)
  #S_index = diction[encode(S)]
  #print(S_index)
  if (time.time() - start) > 10000:
    return 100
  elif (len(S) == 1) & (S[0][0] == v): # Caso base: la soluzione è il peso dell’arco {v, 0}
    #print("1 case")
    return weight(V[v][1], V[0][1], t)
  elif d[v, S_index] != 0: # Distanza già calcolata, ritorna il valore memorizzato
    #print("2 case")
    return d[v, S_index] 
  else:  # Caso ricorsivo: trova il minimo tra tutti i sottocammini
    #print("3 case")
    mindist = math.inf
    minprec = None
    temp = [i for i in S if i[0] != v] # S \ {v} 
    subsets.append(temp) if temp not in subsets else subsets

    if encode(temp) not in diction:
      global counter
      diction[encode(temp)] = counter
      counter += 1

    #print(time.time() - start1)
    for u in temp:
      dist = HK_VISIT(u[0], temp)
      w = weight(u[1], V[v][1], t)
      if (dist + w) < mindist:
        mindist = dist + w
        minprec = u[0]
    #print(v, S_index)
    d[v, S_index] = mindist
    phi[v, S_index] = minprec
    return mindist

time: 28.4 ms


In [252]:
print(HK_VISIT(0, V))
print(time.time() - start , "s")
print(diction)
print(tot)
#print(np.matrix(subsets))
#print(subsets)
print(np.matrix(d))

3323.0
30.62327218055725 s
{'012345678910111213': 0, '12345678910111213': 1, '2345678910111213': 2, '345678910111213': 3, '45678910111213': 4, '5678910111213': 5, '678910111213': 6, '78910111213': 7, '8910111213': 8, '910111213': 9, '10111213': 10, '111213': 11, '1213': 12, '13': 13, '12': 14, '1113': 15, '11': 16, '1112': 17, '101213': 18, '1013': 19, '10': 20, '1012': 21, '101113': 22, '1011': 23, '101112': 24, '9111213': 25, '91213': 26, '913': 27, '9': 28, '912': 29, '91113': 30, '911': 31, '91112': 32, '9101213': 33, '91013': 34, '910': 35, '91012': 36, '9101113': 37, '91011': 38, '9101112': 39, '810111213': 40, '8111213': 41, '81213': 42, '813': 43, '8': 44, '812': 45, '81113': 46, '811': 47, '81112': 48, '8101213': 49, '81013': 50, '810': 51, '81012': 52, '8101113': 53, '81011': 54, '8101112': 55, '89111213': 56, '891213': 57, '8913': 58, '89': 59, '8912': 60, '891113': 61, '8911': 62, '891112': 63, '89101213': 64, '891013': 65, '8910': 66, '891012': 67, '89101113': 68, '891011'